<a href="https://colab.research.google.com/github/CthulhuCoder01/Spam-Mail-Prediction-Custom/blob/main/Spam_Mail_Prediction_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [31]:
from scipy.sparse import csr_matrix

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None
        self.loss_history = []

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, y_pred, y):
        epsilon = 1e-10  # small value to avoid log(0)
        loss = -np.mean(y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon))
        return loss

    def fit(self, X, y):
        num_samples, num_features = X.shape

        # Convert X to dense matrix if it is in csr_matrix format
        if isinstance(X, csr_matrix):
            X = X.toarray()

        # Initialize weights and bias
        self.weights = np.zeros(num_features)
        self.bias = 0

        # Gradient descent optimization
        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            # Calculate loss and append to loss history
            loss = self.compute_loss(y_pred, y)
            self.loss_history.append(loss)

            # Calculate gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        # Convert X to dense matrix if it is in csr_matrix format
        if isinstance(X, csr_matrix):
            X = X.toarray()

        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        y_pred_class = np.where(y_pred > 0.5, 1, 0)
        return y_pred_class


Data Collection & Preprocession

In [7]:
# Loading The data from cssv file to a pandas data frame
raw_mail_data = pd.read_csv('/content/mail_data.csv')

In [8]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [10]:
#Now replace all nul value to string using where operator
# when data row is not null do nothing if null change to ''
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [11]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
#cheack shape of data
mail_data.shape

(5572, 2)

Label encoding change label to numeric outcomes

In [15]:
#cheack every row usin loc funciton
mail_data.loc[mail_data['Category']=='spam', 'Category'] = 0
mail_data.loc[mail_data['Category']=='ham', 'Category'] = 1


In [16]:
mail_data.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


Sperating Text and label into seperate array

In [17]:

X = mail_data['Message']

Y = mail_data['Category']


In [18]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


Splitting into training and testing data

In [24]:
#Ofcourse use testing and train  split funciton
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)


In [25]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


Now comes feature Extraction used to extract feature as numerical data

In [26]:
feature_extraction = TfidfVectorizer( min_df=1, stop_words='english', lowercase=True)

# Tranform Using Feature extracion
X_train_feature = feature_extraction.fit_transform(X_train)

X_test_features = feature_extraction.transform(X_test)


# Convert Y_Train and  Y_Test value to string

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')


In [28]:
print(X_train_feature)

  (0, 3916)	0.6037953148318957
  (0, 3056)	0.3429005562464134
  (0, 4893)	0.6433735801583021
  (0, 3068)	0.32236758935606574
  (1, 5720)	0.6023019309762907
  (1, 6542)	0.5742615392242952
  (1, 3970)	0.2895925989409213
  (1, 6110)	0.4728553638784458
  (2, 53)	0.2275977849591067
  (2, 5502)	0.16144032675968542
  (2, 1582)	0.2275977849591067
  (2, 6557)	0.21161850376233796
  (2, 260)	0.19690190061164803
  (2, 4387)	0.1624258510707269
  (2, 4347)	0.2040320228676743
  (2, 2878)	0.4232962626786592
  (2, 434)	0.2587500279452028
  (2, 2182)	0.27253511506960687
  (2, 4988)	0.1604825674220973
  (2, 1850)	0.21615833183721392
  (2, 453)	0.2587500279452028
  (2, 4630)	0.18805274167090552
  (2, 4587)	0.16079883210369647
  (2, 7099)	0.14954143140130233
  (2, 635)	0.29610087716103933
  :	:
  (4454, 2954)	0.5425177822436338
  (4454, 4823)	0.5172606639045361
  (4454, 3557)	0.5172606639045361
  (4454, 3064)	0.41298579528057155
  (4455, 5046)	0.5672102378436935
  (4455, 5597)	0.5672102378436935
  (4455, 4

Training an model

In [32]:
model = LogisticRegression()

In [33]:
model.fit(X_train_feature,Y_train)

Evaluating Model

In [34]:
#Prediction on training data

prediction_on_training_data = model.predict(X_train_feature)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)



In [36]:
print('Accuracy On Training Data :',accuracy_on_training_data)

Accuracy On Training Data : 0.8649315683194975


In [37]:
# Prediction on test data


prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)


In [38]:
print('Accuracy On test Data :',accuracy_on_test_data)

Accuracy On test Data : 0.8699551569506726


Final is to Building a predictive system

In [40]:
input_mail = [
    "Nah I don't think he goes to usf, he lives around here though",
    "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
]

# Convert mail text into featurevalue

input_main_features =  feature_extraction.transform(input_mail)

# Making Presictions

prediction = model.predict(input_main_features)

print(prediction)

if prediction[0] == 1 :
  print("Legit mail")
else:
  print("Spam Mail")

if prediction[1] == 1 :
  print("Legit mail Two")
else:
  print("Spam Mail Two")

[1 1]
Legit mail
Legit mail Two
